 <img src="https://raw.githubusercontent.com/scalabrinig/cdProjetoAplicadoIV/d093146488f56dfcf0ef286bcee8efe0e71b9c76/figuras/mackenzie_logo.jpg" width="25%" align="right"/>

# **PROJETO APLICADO IV - Ciência de Dados EaD - 2025/02**


# **Código**

---


# **Titulo do Projeto**: Projeto Aplicado IV – Análise de séries temporais para descrição e previsões

In [1]:
#@title **Identificação do Grupo e Opção do Projeto**

#@markdown Integrantes do Grupo, nome completo em ordem alfabética (*informe: \<nome\>, \<matrícula\>*)
Aluno1 = 'Carlos Oliveira, 10424072' #@param {type:"string"}
Aluno2 = 'Erick Isidoro, 10424337' #@param {type:"string"}
Aluno3 = 'Felipe Perraz, 10218996' #@param {type:"string"}

Link do Dados 

https://www.kaggle.com/code/leandroal/an-lise-do-e-commerce-no-brasil-olist-dataset

1. Introdução 
No Brasil, milhares de compras são efetuadas pela Internet todos os anos. Trata-se de um mercado em constante ritmo de crescimento, de modo que seria extremamente relevante tentar entender comportamentos do consumidor inerentes a este contexto, como meses em que ocorrem maior quantidade de compras, horário em que são realizados mais negócios, tempo médio de entrega, nível de satisfação, etc.

Para isso, foi selecionado um conjunto de dados disponibilizado pela Olist, a maior loja de departmento dos "marketplaces", com informações de mais de 100 mil pedidos de 2016 a 2018.

Como se tratam de dados reais, estes foram disponibilizados de forma a manter o anonimato de qualquer indivíduo, empresa ou parceiro.

## Importação do banco de dados 

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


Importação da Blibliotecas

In [3]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import folium
from folium import plugins
from sklearn.cluster import DBSCAN
print('Bibliotecas preparadas!')

Bibliotecas preparadas!


Importação dos Datasets
Foi realizada a importação de todos os datasets através do Pandas e posteriormente foram cruzados a fim de formar um único Dataframe com informações de: consumidores, geolocalização, pedidos, artigos, produtos, vendedores e avaliações.

In [4]:
files = {
    'customers': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_customers_dataset.csv',
    'geolocation': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_geolocation_dataset.csv',
    'items': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_order_items_dataset.csv',
    'payment': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_order_payments_dataset.csv',
    'orders': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_orders_dataset.csv',
    'products': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_products_dataset.csv',
    'sellers': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_sellers_dataset.csv',
    'review': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_order_reviews_dataset.csv',
}


dfs = {}
for key, value in files.items():
    dfs[key] = pd.read_csv(value)

# Exibir as primeiras linhas de um dataset (exemplo: customers)
print(dfs['customers'].head())



                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  


Cruzamento entre os Datasets e Transformação de Dados
Os cruzamentos foram realizados de forma gradativa, para assegurar que não ocorresse duplicação de dados.


In [5]:
# Cruzamento gradativo
customers_location = dfs['customers'].merge(dfs['geolocation'], how='inner', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix').drop_duplicates('customer_id', keep='first')
cusloc_order = customers_location.merge(dfs['orders'], how='inner', on='customer_id')
cuslocord_item = cusloc_order.merge(dfs['items'], how='inner', on='order_id')
cuslocordite_prod = cuslocord_item.merge(dfs['products'], how='inner', on='product_id')
cuslocordite_rev= cuslocordite_prod.merge(dfs['review'], how='left', on='order_id')

# Selecionando as colunas de interesse
final = cuslocordite_rev[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state',
       'geolocation_lat', 'geolocation_lng','order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_photos_qty',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp']]


## Tamanho do DF

In [6]:
final.shape

(113007, 28)

## Mostrando os primeiros registros

In [7]:
final.head

<bound method NDFrame.head of                              customer_id                customer_unique_id  \
0       06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1       18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2       4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3       b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4       4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   
...                                  ...                               ...   
113002  17ddf5dd5d51696bb3d7c6291687be6f  1a29b476fee25c95fbafc67c5ac95cf8   
113003  e7b71a9017aa05c9a7fd292d714858e8  d52a67c98be1cf6a5c84435bd38d095d   
113004  5e28dfe12db7fb50a4b2f691faecea5e  e9f50caf99f032f0bf3c55141f019d99   
113005  56b18e2166679b8a959d72dd06da27f9  73c2643a0a458b49f58cea58833b192e   
113006  274fa6071e5e17fe303b9748641082c8  84732c5050c01db9b23e19ba39899398   

        customer_zip_code_prefix 

## Todas as Colunas

In [8]:
final.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'geolocation_lat', 'geolocation_lng',
       'order_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name', 'product_photos_qty',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

Também tomou-se a decisão de remover dados do ano de 2016 e do mês setembro de 2018, o primeiro por conter apenas 375 registros e o segundo por conter apenas um registro.

In [9]:
# Garantindo que todas as colunas sejam datetime
datas = [
    'order_purchase_timestamp',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'shipping_limit_date',
    'review_creation_date',
    'review_answer_timestamp'
]

for data in datas:
    final[data] = pd.to_datetime(final[data], errors='coerce')  # Converter e tratar erros

# Criando coluna de tempo de entrega
final['delivery_time'] = (final['order_delivered_customer_date'] - final['order_purchase_timestamp']).dt.days

# Seleção do período de interesse
final = final[(final['order_purchase_timestamp'].dt.year > 2016) 
              & 
              (final['order_purchase_timestamp'] < pd.to_datetime('20180901'))
             ]
final = final.reset_index(drop=True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112632 entries, 0 to 112631
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_id                    112632 non-null  object        
 1   customer_unique_id             112632 non-null  object        
 2   customer_zip_code_prefix       112632 non-null  int64         
 3   customer_city                  112632 non-null  object        
 4   customer_state                 112632 non-null  object        
 5   geolocation_lat                112632 non-null  float64       
 6   geolocation_lng                112632 non-null  float64       
 7   order_id                       112632 non-null  object        
 8   order_status                   112632 non-null  object        
 9   order_purchase_timestamp       112632 non-null  datetime64[ns]
 10  order_approved_at              112617 non-null  object        
 11  

## Tamanho do DF Removidos valores 

In [10]:
final.shape

(112632, 29)

customer_id: Identificação única do cliente (específica para o sistema).

customer_unique_id: Identificador exclusivo do cliente, usado para rastrear o mesmo cliente em várias transações.

customer_zip_code_prefix: Prefixo do código postal do cliente.

customer_city: Cidade onde o cliente está localizado.

customer_state: Estado onde o cliente está localizado.

geolocation_lat: Latitude do cliente (geolocalização).

geolocation_lng: Longitude do cliente (geolocalização).

order_id: Identificação única do pedido.

order_status: Status do pedido (por exemplo, entregue, pendente, cancelado).

order_purchase_timestamp: Data e hora em que o pedido foi realizado.

order_approved_at: Data e hora em que o pedido foi aprovado.

order_delivered_carrier_date: Data em que o pedido foi enviado ao transportador.

order_delivered_customer_date: Data em que o pedido foi entregue ao cliente.

order_estimated_delivery_date: Data estimada de entrega do pedido.

order_item_id: Identificador único de um item dentro de um pedido (caso tenha múltiplos itens).

product_id: Identificador único do produto.

seller_id: Identificação única do vendedor que forneceu o produto.

shipping_limit_date: Data limite para envio do produto ao transportador.

price: Preço do produto.

freight_value: Valor do frete para o pedido.

product_category_name: Nome da categoria do produto.

product_photos_qty: Número de fotos do produto disponíveis.

review_id: Identificação única da avaliação feita pelo cliente.

review_score: Pontuação da avaliação (exemplo: de 1 a 5 estrelas).

review_comment_title: Título do comentário da avaliação.

review_comment_message: Mensagem detalhada do comentário da avaliação.

review_creation_date: Data em que o cliente criou a avaliação.

review_answer_timestamp: Data em que a avaliação foi respondida pelo sistema ou vendedor.

delivery_time: Tempo de entrega em dias (calculado com base nas datas de compra e entrega).

In [11]:
final.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'geolocation_lat', 'geolocation_lng',
       'order_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name', 'product_photos_qty',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'delivery_time'],
      dtype='object')